# Лабораторная работа No4
## Задача:
### 1. Адаптируйте pizza-скрипт для базы данных PostgreSQL.
Скрип аналогичен скрипту из лабораторной работы No2.
Если есть необходимость, то вы можете применить его заново, предварительно удалив таблицы командой.  
![Диаграмма](media/lab_2_1.png)
#### Таблица `pizzeria` (Справочник пиццерий - ресторанов)
- поле `id` - первичный ключ
- поле `name` - имя пиццы
- поле `rating` - средний рейтинг ресторана (от 0 до 5 звезд)
#### Таблица `person` (Справочник людей кто любит пиццу)
- поле `id` - первичный ключ
- поле `name` - имя человека
- поле `age` - возраст
- поле `gender` - пол
- поле `address` - адрес проживания
#### Таблица `menu` (Справочник доступных меню с ценами и конкретными видами
пицц)
- поле `id` - первичный ключ
- поле `pizzeria_id` - ссылка на пиццерию
- поле `pizza_name` - название пиццы
- поле `price` - цена пиццы
#### Таблица `person_visits` (Операционная таблица с информацией о посещении
людей конкретных пиццерий)
- поле `id` - первичный ключ
- поле `person_id` - ссылка на посетившего человека
- поле `pizzeria_id` - ссылка на ресторан
- поле `visit_date` - дата (например 2022-01-01) визита
#### Таблица `person_order` (Операционная таблица с информацией о покупках пицц
людьми)
- поле `id` - первичный ключ
- поле `person_id` - ссылка на человека, который приобрел пиццу
- поле `menu_id` - ссылка на меню
- поле `order_date` - дата (например 2022-01-01) покупки

### 2. Необходимо написать SQL запросы к следующим 10 задачам ниже.
Задание считается выполненным - если SQL запрос написан синтаксически корректно и возвращает ожидаемые данные на условие задачи.
- Найти все возможные комбинации людей и пиццерий, чтобы понять потенциальные варианты посещений.
- Используя оператор `NATURAL JOIN`, написать SQL запрос для получения список пицц и их цен, которые производятся в пиццериях
- Вывести список людей и их заказы с деталями пиццы.
- Используя оператор `EXISTS`, найти людей, которые сделали заказы в пиццериях с рейтингом выше 4.5.
- Вывести всех клиентов и их соответствующие посещения пиццерий, даже если клиент не посещал пиццерию
- Вывести все пиццерии и количество посещений по каждой, даже если посещений не было
- Используя оператор `LATERAL JOIN`, для каждого человека вывести 3 самых дорогих пиццы, которые он заказывал.
- Используя оператор `LATERAL JOIN`, Для каждой пиццерии найти 2 самых популярных пиццы (по количеству заказов) и показать их названия с количеством заказов.
- Используя оператор `WITH` (CTE), выведите уникальные семантические пары клиентов, родной город которых один и тот же. Результат необходимо отсортировать по первому столбцу. Семантически эквивалентная пара является пара клиентов например (Иванов, Петров) = (Петров, Иванов), в этом случае должна быть выведена одна из пар.

## Создаем подключение к БД

In [54]:
import psycopg2
from tabulate import tabulate

### Подключение к PostgreSQL
connection = psycopg2.connect(
    host="localhost",
    port="5432",
    database="",
    user="",
    password="",
)
### Создание курсора
cursor = connection.cursor()

print("Подключение к PostgreSQL успешно.")

Подключение к PostgreSQL успешно.


## Создание схемы
### !Отключено, для данной работы схема создается в скрипте pizza_model.sql

In [55]:
#cursor.execute("CREATE SCHEMA IF NOT EXISTS ivan_patakin;")
#connection.commit()

#print("Схема 'ivan_patakin' создана.")

## Выполняем pizza-скрипт для создания таблиц

pizza_model был немного изменен в lab_2:
1. Добавлено создание схемы, если она не создана.
2. Добавлен автоинкремент ко всем таблицам и восстановлена последовательность в соответствии с данными уже находящимися в таблице

In [56]:
with open('Data/Lab2/pizza_model.sql', 'r', encoding='utf-8') as file:
    sql_script = file.read()
    
cursor.execute(sql_script)

connection.commit()

print("pizza-скрипт выполнен")

pizza-скрипт выполнен


## SQL запросы:

### SQL запрос, все возможные комбинации людей и пиццерий, чтобы понять потенциальные варианты посещений.

In [57]:
cursor.execute("""
    SELECT 
        person.name AS person_name, 
        pizzeria.name AS pizzeria_name
    FROM ivan_patakin.person
    CROSS JOIN ivan_patakin.pizzeria
    ORDER BY person.name, pizzeria.name;
""")
rows = cursor.fetchall()
print("Все возможные комбинации людей и пиццерий:")
headers = ["Имя", "Название пиццерии"]
print(tabulate(rows, headers=headers, tablefmt="grid"))

Все возможные комбинации людей и пиццерий:
+---------+---------------------+
| Имя     | Название пиццерии   |
+=========+=====================+
| Andrey  | Best Pizza          |
+---------+---------------------+
| Andrey  | DinoPizza           |
+---------+---------------------+
| Andrey  | DoDo Pizza          |
+---------+---------------------+
| Andrey  | Dominos             |
+---------+---------------------+
| Andrey  | Papa Johns          |
+---------+---------------------+
| Andrey  | Pizza Hut           |
+---------+---------------------+
| Anna    | Best Pizza          |
+---------+---------------------+
| Anna    | DinoPizza           |
+---------+---------------------+
| Anna    | DoDo Pizza          |
+---------+---------------------+
| Anna    | Dominos             |
+---------+---------------------+
| Anna    | Papa Johns          |
+---------+---------------------+
| Anna    | Pizza Hut           |
+---------+---------------------+
| Denis   | Best Pizza          |
+----

### SQL запрос, используя оператор `NATURAL JOIN` для получения список пицц и их цен, которые производятся в пиццериях

In [58]:
cursor.execute("""
    SELECT pizza_name, price
    FROM ivan_patakin.menu
    NATURAL JOIN ivan_patakin.pizzeria;
""")
rows = cursor.fetchall()
print("Список пицц и их цен:")
headers = ["Название пиццы", "Цена"]
print(tabulate(rows, headers=headers, tablefmt="grid"))

Список пицц и их цен:
+------------------+--------+
| Название пиццы   |   Цена |
+==================+========+
| cheese pizza     |    900 |
+------------------+--------+
| pepperoni pizza  |   1200 |
+------------------+--------+
| sausage pizza    |   1200 |
+------------------+--------+
| supreme pizza    |   1200 |
+------------------+--------+
| cheese pizza     |    950 |
+------------------+--------+
| pepperoni pizza  |    800 |
+------------------+--------+


### SQL запрос, список людей и их заказы с деталями пиццы

In [59]:
cursor.execute("""
    SELECT 
        person.name AS person_name, 
        menu.pizza_name, 
        menu.price, 
        pizzeria.name AS pizzeria_name,
        person_order.order_date
    FROM ivan_patakin.person
    JOIN ivan_patakin.person_order ON person.id = person_order.person_id
    JOIN ivan_patakin.menu ON person_order.menu_id = menu.id
    JOIN ivan_patakin.pizzeria ON menu.pizzeria_id = pizzeria.id;
""")
rows = cursor.fetchall()
print("Список людей, их заказы с деталями пиццы и названием пиццерии:")
headers = ["Имя", "Название пиццы", "Цена", "Название пиццерии", "Дата заказа"]
print(tabulate(rows, headers=headers, tablefmt="grid"))

Список людей, их заказы с деталями пиццы и названием пиццерии:
+---------+------------------+--------+---------------------+---------------+
| Имя     | Название пиццы   |   Цена | Название пиццерии   | Дата заказа   |
+=========+==================+========+=====================+===============+
| Anna    | cheese pizza     |    900 | Pizza Hut           | 2022-01-01    |
+---------+------------------+--------+---------------------+---------------+
| Anna    | pepperoni pizza  |   1200 | Pizza Hut           | 2022-01-01    |
+---------+------------------+--------+---------------------+---------------+
| Andrey  | cheese pizza     |    800 | Dominos             | 2022-01-01    |
+---------+------------------+--------+---------------------+---------------+
| Andrey  | mushroom pizza   |   1100 | Dominos             | 2022-01-01    |
+---------+------------------+--------+---------------------+---------------+
| Kate    | cheese pizza     |    700 | Best Pizza          | 2022-01-04    |
+

### SQL запрос, используя оператор EXISTS, найти людей, которые сделали заказы в пиццериях с рейтингом выше 4.5.

In [60]:
cursor.execute("""
    SELECT DISTINCT 
        person.name AS person_name,
        pizzeria.name AS pizzeria_name,
        pizzeria.rating AS pizzeria_rating
    FROM ivan_patakin.person
    JOIN ivan_patakin.pizzeria
    ON EXISTS (
        SELECT 1
        FROM ivan_patakin.person_order
        JOIN ivan_patakin.menu ON person_order.menu_id = menu.id
        WHERE person_order.person_id = person.id AND menu.pizzeria_id = pizzeria.id AND pizzeria.rating > 4.5
    );
""")
rows = cursor.fetchall()
print("Люди, которые сделали заказы в пиццериях с рейтингом выше 4.5:")
headers = ["Имя", "Название пиццерии", "Рейтинг пиццерии"]
print(tabulate(rows, headers=headers, tablefmt="grid"))

Люди, которые сделали заказы в пиццериях с рейтингом выше 4.5:
+--------+---------------------+--------------------+
| Имя    | Название пиццерии   |   Рейтинг пиццерии |
+========+=====================+====================+
| Anna   | Pizza Hut           |                4.6 |
+--------+---------------------+--------------------+
| Irina  | Papa Johns          |                4.9 |
+--------+---------------------+--------------------+
| Peter  | Pizza Hut           |                4.6 |
+--------+---------------------+--------------------+
| Nataly | Papa Johns          |                4.9 |
+--------+---------------------+--------------------+


### SQL запрос, всех клиентов и их соответствующие посещения пиццерий, даже если клиент не посещал пиццерию

In [61]:
cursor.execute("""
    SELECT 
        person.name AS person_name, 
        pizzeria.name AS pizzeria_name, 
        person_visits.visit_date
    FROM ivan_patakin.person
    CROSS JOIN ivan_patakin.pizzeria
    LEFT JOIN ivan_patakin.person_visits 
        ON person.id = person_visits.person_id 
        AND pizzeria.id = person_visits.pizzeria_id
    ORDER BY person.name, pizzeria.name;
""")
rows = cursor.fetchall()
print("Все клиенты и их посещения пиццерий (включая тех, кто не посещал):")
headers = ["Имя", "Название пиццерии", "Дата посещения"]
print(tabulate(rows, headers=headers, tablefmt="grid"))

Все клиенты и их посещения пиццерий (включая тех, кто не посещал):
+---------+---------------------+------------------+
| Имя     | Название пиццерии   | Дата посещения   |
+=========+=====================+==================+
| Andrey  | Best Pizza          |                  |
+---------+---------------------+------------------+
| Andrey  | DinoPizza           |                  |
+---------+---------------------+------------------+
| Andrey  | DoDo Pizza          |                  |
+---------+---------------------+------------------+
| Andrey  | Dominos             | 2022-01-01       |
+---------+---------------------+------------------+
| Andrey  | Papa Johns          |                  |
+---------+---------------------+------------------+
| Andrey  | Pizza Hut           | 2022-01-02       |
+---------+---------------------+------------------+
| Anna    | Best Pizza          |                  |
+---------+---------------------+------------------+
| Anna    | DinoPizza           

### SQL запрос, все пиццерии и количество посещений по каждой, даже если посещений не было

In [62]:
cursor.execute("""
    SELECT 
        pizzeria.name AS pizzeria_name, 
        COUNT(person_visits.id) AS visit_count
    FROM ivan_patakin.pizzeria
    LEFT JOIN ivan_patakin.person_visits ON pizzeria.id = person_visits.pizzeria_id
    GROUP BY pizzeria.name;
""")
rows = cursor.fetchall()
print("Все пиццерии и количество посещений по каждой:")
headers = ["Название пиццерии", "Количество посещений"]
print(tabulate(rows, headers=headers, tablefmt="grid"))

Все пиццерии и количество посещений по каждой:
+---------------------+------------------------+
| Название пиццерии   |   Количество посещений |
+=====================+========================+
| DinoPizza           |                      4 |
+---------------------+------------------------+
| Pizza Hut           |                      4 |
+---------------------+------------------------+
| Dominos             |                      5 |
+---------------------+------------------------+
| Best Pizza          |                      3 |
+---------------------+------------------------+
| DoDo Pizza          |                      0 |
+---------------------+------------------------+
| Papa Johns          |                      3 |
+---------------------+------------------------+


### SQL запрос, используя оператор LATERAL JOIN, для каждого человека вывести 3 самых дорогих пиццы, которые он заказывал

In [63]:
cursor.execute("""
    SELECT
        person.name AS person_name,
        top_pizzas.pizza_name,
        top_pizzas.price
    FROM ivan_patakin.person
    LEFT JOIN LATERAL (
        SELECT
            menu.pizza_name,
            menu.price
        FROM ivan_patakin.person_order
        JOIN ivan_patakin.menu ON person_order.menu_id = menu.id
        WHERE person_order.person_id = person.id
        ORDER BY menu.price DESC
        LIMIT 3
    ) AS top_pizzas ON true
    ORDER BY person_name, top_pizzas.price;
""")
rows = cursor.fetchall()
print("3 самых дорогих пиццы для каждого человека:")
headers = ["Имя", "Название пиццы", "Цена"]
print(tabulate(rows, headers=headers, tablefmt="grid"))

3 самых дорогих пиццы для каждого человека:
+---------+------------------+--------+
| Имя     | Название пиццы   |   Цена |
+=========+==================+========+
| Andrey  | cheese pizza     |    800 |
+---------+------------------+--------+
| Andrey  | mushroom pizza   |   1100 |
+---------+------------------+--------+
| Anna    | cheese pizza     |    900 |
+---------+------------------+--------+
| Anna    | pepperoni pizza  |   1200 |
+---------+------------------+--------+
| Denis   | pepperoni pizza  |    800 |
+---------+------------------+--------+
| Denis   | supreme pizza    |    850 |
+---------+------------------+--------+
| Denis   | sausage pizza    |   1000 |
+---------+------------------+--------+
| Dmitriy | pepperoni pizza  |    800 |
+---------+------------------+--------+
| Dmitriy | supreme pizza    |    850 |
+---------+------------------+--------+
| Elvira  | pepperoni pizza  |    800 |
+---------+------------------+--------+
| Elvira  | sausage pizza    |   100

### SQL запрос, используя оператор LATERAL JOIN, Для каждой пиццерии найти 2 самых популярных пиццы (по количеству заказов) и показать их названия с количеством заказов.

In [64]:
cursor.execute("""
    SELECT 
        pizzeria.name AS pizzeria_name, 
        popular_pizzas.pizza_name, 
        popular_pizzas.order_count
    FROM ivan_patakin.pizzeria
    INNER JOIN LATERAL (
        SELECT menu.pizza_name, COUNT(person_order.id) AS order_count
        FROM ivan_patakin.menu
        INNER JOIN ivan_patakin.person_order ON menu.id = person_order.menu_id
        WHERE menu.pizzeria_id = pizzeria.id
        GROUP BY menu.pizza_name
        ORDER BY order_count DESC
        LIMIT 2
    ) AS popular_pizzas ON true;
""")
rows = cursor.fetchall()
print("2 самых популярных пиццы для каждой пиццерии (только с заказами):")
headers = ["Название пиццерии", "Название пиццы", "Количество заказов"]
print(tabulate(rows, headers=headers, tablefmt="grid"))

2 самых популярных пиццы для каждой пиццерии (только с заказами):
+---------------------+------------------+----------------------+
| Название пиццерии   | Название пиццы   |   Количество заказов |
+=====================+==================+======================+
| Pizza Hut           | cheese pizza     |                    1 |
+---------------------+------------------+----------------------+
| Pizza Hut           | pepperoni pizza  |                    1 |
+---------------------+------------------+----------------------+
| Dominos             | cheese pizza     |                    2 |
+---------------------+------------------+----------------------+
| Dominos             | mushroom pizza   |                    2 |
+---------------------+------------------+----------------------+
| Papa Johns          | mushroom pizza   |                    1 |
+---------------------+------------------+----------------------+
| Papa Johns          | pepperoni pizza  |                    1 |
+---------

### SQL запрос, используя оператор WITH (CTE), выведите уникальные семантические пары клиентов, родной город которых один и тот же. Результат необходимо отсортировать по первому столбцу.

In [65]:
cursor.execute("""
    WITH paired_clients AS (
        SELECT 
            p1.name AS client1, 
            p2.name AS client2,
            p1.address AS city
        FROM ivan_patakin.person p1
        JOIN ivan_patakin.person p2 ON p1.address = p2.address AND p1.id < p2.id
    )
    SELECT client1, client2, city
    FROM paired_clients
    ORDER BY client1;
""")
rows = cursor.fetchall()
print("Уникальные семантические пары клиентов с указанием города:")
headers = ["Клиент 1", "Клиент 2", "Город"]
print(tabulate(rows, headers=headers, tablefmt="grid"))

Уникальные семантические пары клиентов с указанием города:
+------------+------------+------------------+
| Клиент 1   | Клиент 2   | Город            |
+============+============+==================+
| Anna       | Andrey     | Moscow           |
+------------+------------+------------------+
| Denis      | Elvira     | Kazan            |
+------------+------------+------------------+
| Irina      | Peter      | Saint-Petersburg |
+------------+------------+------------------+
| Kate       | Elvira     | Kazan            |
+------------+------------+------------------+
| Kate       | Denis      | Kazan            |
+------------+------------+------------------+


## Удаление таблиц

In [66]:
cursor.execute("""
    DROP TABLE IF EXISTS ivan_patakin.person_order;
    DROP TABLE IF EXISTS ivan_patakin.menu;
    DROP TABLE IF EXISTS ivan_patakin.person_visits;
    DROP TABLE IF EXISTS ivan_patakin.pizzeria;
    DROP TABLE IF EXISTS ivan_patakin.person;
""")

connection.commit()

print("Таблицы удалены")

Таблицы удалены


### Закрытие курсора и соединения

In [67]:
cursor.close()
connection.close()

print("Соединение с PostgreSQL закрыто.")

Соединение с PostgreSQL закрыто.
